In [1]:
from redis.cluster import RedisCluster
import faker as fk
import pandas
import random
fake = fk.Faker()

rc = RedisCluster(host='localhost', port=6381, decode_responses=True)

In [2]:
# r.flushdb()
rc.flushdb()

{'127.0.0.1:6381': True, '127.0.0.1:6382': True, '127.0.0.1:6383': True}

In [3]:
def make_rand_computer():
    os = ['windows', 'macOS', 'linux']
    return {"ip": fake.ipv4(), "name": fake.user_name(), "os": random.choice(os)}

def make_rand_mac():
    return fake.mac_address()

def unravel_dict(dictionary):
    return " ".join([f"{key} {value}"for key,value in dictionary.items()])

def add_computer(computer):
    mac = make_rand_mac()
    # r.execute_command(f"HSET mac:{mac} {unravel_dict(computer)}")
    rc.hset(f"mac:{mac}", mapping=computer)
    # r.execute_command(f"HSET index.mac.by.ip {computer['ip']} {mac}")
    rc.hset("index.mac.by.ip", computer['ip'], mac)

    # macs = r.execute_command("HGETALL index.mac.by.ip")
    macs = rc.hgetall("index.mac.by.ip")
    macs = list(macs.values())
    macs.remove(mac)
    for _ in range(4):
        if macs == []:
            break
        connected = random.choice(macs)
        # r.execute_command(f"SADD connections:{mac} {connected}")
        rc.sadd(f"connections:{mac}", connected)
        # r.execute_command(f"SADD connections:{connected} {mac}")
        rc.sadd(f"connections:{connected}", mac)

def initialize_content(n):
    for _ in range(n):
        add_computer(make_rand_computer())


In [4]:
def show_all_macs():
    # for key in r.scan_iter("mac:*"):
    for key in rc.scan_iter("mac:*"):
        print(key)

def show_all_computers():
    # for key in r.scan_iter("mac:*"):
    for key in rc.scan_iter("mac:*"):
        print(key)
        # print(r.execute_command(f"HGETALL {key}"))
        print(rc.hgetall(key))

def show_all_connections():
    # for key in r.scan_iter("mac:*"):
    for key in rc.scan_iter("mac:*"):
        mac = key[len("mac:"):]
        print(mac)
        # print(r.execute_command(f"SMEMBERS connections:{mac}"))
        print(rc.smembers(f"connections:{mac}"))

def change_ip(old_ip, new_ip):
    # r.execute_command(f"WATCH index.mac.by.ip")
    rc.watch("index.mac.by.ip")
    # mac_address = r.execute_command(f"HGET index.mac.by.ip {old_ip}")
    mac_address = rc.hget("index.mac.by.ip", old_ip)
    # r.execute_command(f"MULTI")
    # pass
    # r.execute_command(f"HSET index.mac.by.ip {new_ip} {mac_address}")
    rc.hset("index.mac.by.ip", new_ip, mac_address)
    # r.execute_command(f"HDEL index.mac.by.ip {old_ip}")
    rc.hdel("index.mac.by.ip", old_ip)
    # r.execute_command(f"HSET mac:{mac_address} ip {new_ip}")
    rc.hset(f"mac:{mac_address}", "ip", new_ip)
    # r.execute_command('PUBLISH', 'computer.logs', f"IP of computer {mac_address} has changed from {old_ip} to {new_ip}")
    rc.publish('computer.logs', f"IP of computer {mac_address} has changed from {old_ip} to {new_ip}")
    # r.execute_command(f"EXEC")
    rc.unwatch()

In [5]:
initialize_content(10)

In [6]:
show_all_computers()

mac:5a:d8:7c:d3:eb:78
{'ip': '1.149.220.114', 'name': 'mark67', 'os': 'windows'}
mac:46:36:b0:33:46:97
{'ip': '109.180.81.112', 'name': 'nortonjustin', 'os': 'linux'}
mac:00:d7:9f:ff:32:b9
{'ip': '22.234.208.5', 'name': 'justin44', 'os': 'windows'}
mac:58:a9:f7:82:82:db
{'ip': '116.29.164.41', 'name': 'colleendickerson', 'os': 'linux'}
mac:fc:01:6e:9d:f9:b1
{'ip': '151.216.26.124', 'name': 'zpace', 'os': 'macOS'}
mac:b8:a7:2e:b4:b7:f1
{'ip': '193.113.71.237', 'name': 'samanthabaker', 'os': 'windows'}
mac:d0:35:a1:97:b6:ff
{'ip': '158.139.217.164', 'name': 'curtiscameron', 'os': 'macOS'}
mac:0c:2c:e1:49:6c:a9
{'ip': '44.34.174.156', 'name': 'brandtpatrick', 'os': 'linux'}
mac:04:90:11:8c:9a:53
{'ip': '66.222.100.43', 'name': 'wellsaustin', 'os': 'macOS'}
mac:48:3f:68:5e:0c:69
{'ip': '148.109.235.197', 'name': 'lucasjoseph', 'os': 'linux'}


In [7]:
key = next(rc.scan_iter("mac:*"))
computer_dict = rc.hgetall(key)
print(computer_dict)
ip_to_change = computer_dict['ip']

{'ip': '1.149.220.114', 'name': 'mark67', 'os': 'windows'}


In [8]:
change_ip(ip_to_change,"119.22.49.0")

In [9]:
computer_dict = rc.hgetall(key)
print(computer_dict)
ip_to_change = computer_dict['ip']

{'ip': '119.22.49.0', 'name': 'mark67', 'os': 'windows'}
